In [ ]:
#Libraries
try:
    import openfermion as of
except ImportError:
    !pip install -q git+https://github.com/quantumlib/OpenFermion.git@master#egg=openfermion 
    import openfermion as of

try:
    import cirq
except ImportError:
    !pip install cirq --quiet
    import cirq

try:
    import fqe
except ImportError:
    !pip install git+https://github.com/quantumlib/OpenFermion-FQE --quiet
    import fqe

from openfermion import FermionOperator
import numpy as np
from scipy.sparse import csr_matrix


# Comparing OpenFermion and "by hand" Hamiltonian
We proceed with $N = 4$ fermions, to print matrices without filling up the RAM.
Openfermion has the module *get_sparse_operator* which converts from the annihilation and creation operator basis to a matrix on the qubits Hilbert space.
This means we should be able to obtain the same Hamiltonian "by hand", using tensor products: this has been verified, and now *assert* statements replace prints and other tests.

In [ ]:
N = 4
N_qubits = 2*N

a = np.matrix(([0, 1], [0, 0]))
a_dagger = a.T
sigma_z = np.matrix(([1,0], [0,-1]))

kill_op = [] #will contain annihilation operators for each site

# a_1 = a ⊗ I(2**(N_qubits-1)), a_2 = sigma_z ⊗ a ⊗ I(2**(N_qubits -1 -1)), ..., a_i = sigma_z^i ⊗ a ⊗ I(2**(N_qubits -1 -i))

for ii in range(N_qubits):
  operator = a
  # sigma_z^i ⊗ a, adding sigma_z to the left i times
  for jj in range(ii):
    operator = np.kron(sigma_z, operator)
  #I(2**N-1-i) to the right
  operator = np.kron(operator, np.eye(2**(N_qubits-1-ii))) 
  assert(operator.shape[0] == 2**(N_qubits)) #check dimension

  kill_op.append(operator)

#create a list of creation operators in matrix form
born_op = []
for el in kill_op:
  born_op.append(el.H) #no need for Hermitian conjugate in this representation
  
n_op = [c * d for (c,d) in zip(born_op, kill_op)] #c = create, d = destruct

#check n1
n1_to_check = np.kron(a_dagger @ a, np.eye(2**(N_qubits -1)))
assert((n_op[0] == n1_to_check).all())

#converting to sparse
kill_op = [csr_matrix(el) for el in kill_op]
born_op = [csr_matrix(el) for el in born_op]
n_op = [csr_matrix(el) for el in n_op]

#building the Hamiltonian
J = 1
H_J = []

#sum_j sum_nu for j in [1, L-1], with L = N sites, separating nu = up from down
for i in range(0, 2*N - 2 , 2):
  H_J.append(-J*born_op[i] @ kill_op[i+2])
  H_J.append(-J*born_op[i+2] @ kill_op[i])   #hermitian conjugated
for i in range(1, 2*N - 2 , 2):
  H_J.append(-J*born_op[i] @ kill_op[i+2])
  H_J.append(-J*born_op[i+2] @ kill_op[i])

hopping_term = csr_matrix.todense(sum(H_J))

#CHECK: matrix coincides with openfermion one
H_J_of = [op + of.hermitian_conjugated(op) for op in (FermionOperator(((i,1), (i+2,0)), coefficient = -J) 
          for i in range(N_qubits - 2))]

assert (of.get_sparse_operator(sum(H_J_of)).todense().real == csr_matrix.todense(sum(H_J)) ).all()

#----------- U matrix--------------
U = 1
H_U = []
for i in range(0,N):
  H_U.append(U*n_op[2*i] @ n_op[2*i+1])

coulomb_term = (csr_matrix.todense(sum(H_U)))

#openfermion version

H_U_of = [op  for op in (FermionOperator(((i,1), (i,0), (i+1, 1), (i+1,0)), coefficient = U)
          for i in range(0, N_qubits, 2))]

assert (of.get_sparse_operator(sum(H_U_of)).todense().real == csr_matrix.todense(sum(H_U))).all()


#-----------------------------------------Interaction potential---------------------------------------------------------------
L = 4
m = 4.5
sigma = 1

#do it only for the up qubits
epsilon = [-4 * np.exp( 0.5 * ((i+1)-m)**2 / sigma**2  ) for i in range(N)]
H_epsilon = []

#construct sum epsilon * n_i on up; n_op has the right expression for considering the whole qubit set (also down, from a tensor prod with I)
for i in range(0, N_qubits, 2):
  H_epsilon.append(epsilon[i//2] * n_op[i])

local_potential =  (csr_matrix.todense(sum(H_epsilon)))
assert(local_potential.shape[0] == 2**N_qubits)

#Here openfermion would produce a smaller Hamiltonian, since it wouldn't act on odd sites


We can also check that Openfermion's own function to build a Fermi-Hubbard Hamiltonian coincides with our construction

In [ ]:
hubbard = of.fermi_hubbard(1, N, tunneling = J, coulomb = U, periodic = False)
assert (sum(H_U_of) + sum(H_J_of) == hubbard)